In [250]:
import pandas as pd;
import numpy as np;
import plotly.express as px
artists=pd.read_csv("data/artists.csv")


# Artist Dataset


In [251]:
artists.sample(5)

,id,followers,genres,name,popularity
332768,2N9RYbFNaumoJTEw67qsBt,212.0,[],So-So Heroes 瘦瘦英雄,9
792491,3QOymHFOLUCitnRiY5czEu,5.0,[],The Next Ones in Line,0
239290,3e2RuJyq5LI5f6EM7SANez,693.0,[],Bouddha,11
623044,0tsXBeeMipHNUtu5GtUAWV,943.0,['k-indie'],onthedal,18
62717,0BXvMNg9gmCaE4nhQnM7n0,3.0,[],Beatmonster Fat,1


# The most popular genre

In [252]:
artists["TmpGenres"]=artists["genres"].apply(lambda x: x.strip('][').split(','))#convert arrays
allGeners=np.unique(np.concatenate(artists["TmpGenres"]))
allGeners[0]="unknown"



In [253]:
import operator
macroGeneres=dict.fromkeys(["alternative","rock","metal","pop","rap","punk","jazz","reggae","soul","polka","country","electronic","funk","hip hop","r&b","folk","house","techno","trance","indie","blues","instrument","electronic","disco","deep","hardcore","wave","other"],0)
totalGener={}
def getMacroCategory(str):
    counts = dict()
    for e in str:
        words = e.split()
        for word in words:
            word=word.replace("'","")
            if word in counts:
                counts[word] += 1
            else:
                counts[word] = 1
    counts={key: value for key, value in sorted(counts.items(), key=lambda item: item[1],reverse=True)}


    for z in counts.keys():

        if z in macroGeneres.keys():

            macroGeneres[z]+=1
            return z
    return "other"


artists["macro genre"]=artists.apply(lambda z:getMacroCategory(z["TmpGenres"]),axis=1)


In [263]:
generesDf={'Generes':macroGeneres.keys(),'Number':macroGeneres.values()}

macroGeneresDF=pd.DataFrame.from_dict(generesDf)
macroGeneresDF=macroGeneresDF.sort_values(by="Number",ascending=False)


In [277]:
fig=px.histogram(macroGeneresDF,y="Generes",x="Number")
fig.update_yaxes(categoryorder="total ascending")
fig.layout["xaxis"]["title"]="Generes"
fig.layout["yaxis"]["title"]="Number of Artists"
fig.layout.yaxis.dtick=0.5
fig.show()

## Relationship between followers and popularity

In [256]:
tmpData=artists.sample(10000);
fig=px.scatter(tmpData,x="followers",y="popularity",hover_name=tmpData["name"], log_x=True)#
fig.show()

## The top 20 of the most popular metal artists

In [257]:

def categorized(g):
    artists.loc[artists["genres"].apply(lambda x : g in x)==True,["macro genre"]]=g
    return artists.loc[artists["macro genre"]==g]


In [258]:
metalArtists=categorized("metal")
top20Metal=metalArtists.sort_values(by="popularity",ascending=False)
fig=px.histogram(top20Metal[:20],x="popularity",y="name")
fig.update_yaxes(categoryorder="total ascending")
fig.layout["yaxis"]["title"]="Band"
fig.layout["xaxis"]["title"]="Popolarity"
fig.show()

# Maximum Popularity per macro-genre

In [259]:
maxPerGenere={}

for a in macroGeneres:
    maxPerGenere[a]=categorized(a).sort_values(by="popularity",ascending=False).iloc[0]["popularity"]




In [278]:

fig=px.histogram(pd.DataFrame.from_dict({'Genere':maxPerGenere.keys(),"Popularity":maxPerGenere.values()}).sort_values(by="Popularity",ascending=False)[1:],x="Genere",y="Popularity")
fig.update_yaxes(categoryorder="total ascending")
fig.layout["yaxis"]["title"]="Popularity"
fig.layout["xaxis"]["title"]="Genre"
fig.show()